In [47]:
# !pip install bareunpy
# !pip install kiwipiepy

In [49]:
import sys
import google.protobuf.text_format as tf
from bareunpy import Tokenizer, Tagger

# You can get an API-KEY from https://bareun.ai/
# Please note that you need to sign up and verify your email.
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"
API_KEY = "koba-GRKAAOA-7DDU2WA-TMZS6VY-AIXYICQ" # <- 본인의 API KEY로 교체(Replace this with your own API KEY)

# If you have your own localhost bareun.
my_tokenizer = Tokenizer(API_KEY, 'localhost')

In [50]:
tagger = Tagger(API_KEY, 'localhost')

In [51]:
res = tagger.tags(['안녕하세요.'])

In [52]:
# print results.
res = tagger.tags(["안녕하세요.", "반가워요!"])

In [53]:
# get tuple of pos tagging.
pa = res.pos()
print(pa)

[('안녕하', 'VA'), ('시', 'EP'), ('어요', 'EF'), ('.', 'SF'), ('반갑', 'VA'), ('어요', 'EF'), ('!', 'SF')]


In [54]:
res = tagger.tags(["안녕하세요.", "반가워요!"])
ma = res.morphs()
print(ma)

['안녕하', '시', '어요', '.', '반갑', '어요', '!']


In [55]:
na = res.nouns()
print(na)

[]


In [56]:
va = res.verbs()
print(va)

[]


In [57]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

In [58]:
from tqdm import tqdm, trange

tqdm.pandas()

In [59]:
train_path = './data/ratings_train.txt'
test_path = './data/ratings_test.txt'

train = pd.read_table(train_path)
test = pd.read_table(test_path)

In [60]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [61]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [62]:
print(f"train shape => {train.shape} \ntest shape => {test.shape}")

train shape => (150000, 3) 
test shape => (50000, 3)


### 최종 전처리 함수
- 한글, 숫자, 초성, ♥♡乃ㄳㅄ의 문자를 남겼다
- 초성 ㅜㅜ, ㅋㅋ이 감정의 의미를 담을 수 있다고 파악했기 때문이다
- 다중 공백을 제거하는 코드를 추가하였다
- ㅋ와 ㅋㅋ은 다르게 인지하였다
    - ㅜㅜㅜ, ㅋㅋㅋㅋㅋㅋ 등의 다중 초성은 2개의 초성으로 모두 합쳐주었다
- 빈 하트와 채워진 하트는 채워진 하트로 통일하고 하트의 갯수는 모두 하나로 바꿔주었다
- 반복되는 문자열은 하나만 남기고 지워줬다

In [63]:
import re

In [64]:
def preprocessing(train, col='document'):
    train[col] = train[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9♥♡乃ㄳㅄ ]","") # 정규 표현식 수행
    train[col] = train[col].str.replace('^ +', "") # 공백은 empty 값으로 변경
    train[col].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
    train.dropna(how='any', inplace=True)
    
    train[col] = train[col].apply(lambda x: re.sub(r'(ㅋ|ㅎ|ㅜ|ㅠ){2,}', lambda m: m.group(1) * 2, x)) # 반복되는 2자리 문자 처리

    train[col] = train[col].str.replace(r'♡','♥')
    train[col] = train[col].apply(lambda x: re.sub(r'♥+', '♥', x))
    train[col] = train[col].apply(lambda x: re.sub(r'\b(\S+)( \1)+', r'\1', x))
    train[col] = train[col].apply(lambda x: ' '.join(x.split()))
    return train

In [65]:
train = preprocessing(train, col='document')
test = preprocessing(test, col='document')

/var/folders/1b/dx8_94fs0q740wx3lfknk6y40000gn/T/ipykernel_92450/2614627157.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train[col] = train[col].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9♥♡乃ㄳㅄ ]","") # 정규 표현식 수행
/var/folders/1b/dx8_94fs0q740wx3lfknk6y40000gn/T/ipykernel_92450/2614627157.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train[col] = train[col].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [66]:
# 훈련 데이터와 테스트 데이터 분리
train_texts, train_labels = train['document'].tolist(), train['label'].tolist()
test_texts, test_labels = test['document'].tolist(), test['label'].tolist()

In [67]:
import os
os.environ["JAVA_HOME"] = '/Library/Java/JavaVirtualMachines/jdk-1.8.jdk/Contents/Home'

In [68]:
train_texts = train_texts[:1000]

In [96]:
import time
from konlpy.tag import Okt, Kkma, Hannanum, Komoran, Mecab
from kiwipiepy import Kiwi

# 토크나이저 초기화
okt = Okt()
kkma = Kkma()
komoran = Komoran()
mecab = Mecab()
kiwi = Kiwi()
hannanum = Hannanum()

# 토크나이저 함수 정의
def tokenize_okt(texts):
    return [okt.pos(text) for text in texts]

def tokenize_hannanum(texts):
    return [hannanum.pos(text) for text in texts]

def tokenize_kkma(texts):
    return [kkma.pos(text) for text in texts]

def tokenize_komoran(texts):
    return [komoran.pos(text) for text in texts]

def tokenize_mecab(texts):
    return [mecab.pos(text) for text in texts]

def tokenize_kiwi(texts):
    return [kiwi.tokenize(text) for text in texts]

def tokenize_bareun(texts):
    bareun = Tagger(API_KEY, 'localhost')
    return [bareun.tags([text]).pos() for text in texts]

def tokenize_okt_norm(texts):
    return [okt.pos(text, norm=True) for text in texts]

def tokenize_okt_norm_stem(texts):
    return [okt.pos(text, stem=True, norm=True) for text in texts]

def tokenize_okt_stem(texts):
    return [okt.pos(text, stem=True) for text in texts]

# 토크나이저 실행 시간 측정
def measure_time(tokenizer, texts, iterations=5):
    total_time = 0
    for _ in range(iterations):
        start_time = time.time()
        tokens = tokenizer(texts)
        end_time = time.time()
        total_time += end_time - start_time
    average_time = total_time / iterations
    return tokens, average_time

# 고정된 조건에서 성능 측정
iterations = 5

# 토큰화 및 실행 시간 측정
train_tokens_okt, time_okt = measure_time(tokenize_okt, train_texts)
print(f"Okt 실행 시간: {time_okt:.2f} 초")

train_tokens_kkma, time_kkma = measure_time(tokenize_kkma, train_texts)
print(f"Kkma 실행 시간: {time_kkma:.2f} 초")

train_tokens_komoran, time_komoran = measure_time(tokenize_komoran, train_texts)
print(f"Komoran 실행 시간: {time_komoran:.2f} 초")

train_tokens_mecab, time_mecab = measure_time(tokenize_mecab, train_texts)
print(f"Mecab 실행 시간: {time_mecab:.2f} 초")

train_tokens_hannanum, time_hannanum = measure_time(tokenize_hannanum, train_texts)
print(f"Hannanum 실행 시간: {time_hannanum:.2f} 초")

train_tokens_kiwi, time_kiwi = measure_time(tokenize_kiwi, train_texts)
print(f"Kiwi 실행 시간: {time_kiwi:.2f} 초")

train_tokens_bareun, time_bareun = measure_time(tokenize_bareun, train_texts)
print(f"bareun 실행 시간: {time_bareun:.2f} 초")

train_tokens_okt_norm, time_okt_norm = measure_time(tokenize_okt_norm, train_texts)
print(f"Okt norm 실행 시간: {time_okt_norm:.2f} 초")

train_tokens_okt_stem, time_okt_stem = measure_time(tokenize_okt_stem, train_texts)
print(f"Okt stem 실행 시간: {time_okt_stem:.2f} 초")

train_tokens_okt_norm_stem, time_okt_norm_stem = measure_time(tokenize_okt_norm_stem, train_texts)
print(f"Okt norm_stem 실행 시간: {time_okt_norm_stem:.2f} 초")

Okt 실행 시간: 1.44 초
Kkma 실행 시간: 12.80 초
Komoran 실행 시간: 0.28 초
Mecab 실행 시간: 0.02 초
Hannanum 실행 시간: 2.21 초
Kiwi 실행 시간: 0.66 초
bareun 실행 시간: 12.16 초
Okt norm 실행 시간: 4.04 초
Okt stem 실행 시간: 2.27 초
Okt norm_stem 실행 시간: 4.04 초


In [84]:
train_tokens_okt[0]

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나네요', 'Adjective'),
 ('목소리', 'Noun')]

In [85]:
train_tokens_kkma[0]

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [86]:
train_tokens_komoran[0]

[('아', 'IC'),
 ('더빙', 'NNP'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'VV'),
 ('네요', 'EC'),
 ('목소리', 'NNG')]

In [87]:
train_tokens_hannanum[0]

[('아', 'I'),
 ('더빙', 'N'),
 ('진짜', 'N'),
 ('짜증나', 'P'),
 ('네', 'E'),
 ('요', 'J'),
 ('목소리', 'N')]

In [88]:
train_tokens_kiwi[0]

[Token(form='아', tag='IC', start=0, len=1),
 Token(form='더빙', tag='NNG', start=2, len=2),
 Token(form='진짜', tag='MAG', start=5, len=2),
 Token(form='짜증', tag='NNG', start=8, len=2),
 Token(form='나', tag='VV', start=10, len=1),
 Token(form='네요', tag='EF', start=11, len=2),
 Token(form='목소리', tag='NNG', start=14, len=3)]

In [89]:
train_tokens_mecab[0]

[('아', 'IC'),
 ('더', 'MAG'),
 ('빙', 'MAG'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'NP'),
 ('네요', 'VCP+EC'),
 ('목소리', 'NNG')]

In [94]:
train_tokens_bareun[0]

[('아', 'IC'),
 ('더빙', 'IC'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'VV'),
 ('네', 'EF'),
 ('요', 'JX'),
 ('목소리', 'NNG')]

In [91]:
train_tokens_okt_norm[0]

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나네요', 'Adjective'),
 ('목소리', 'Noun')]

In [92]:
train_tokens_okt_stem[0]

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나다', 'Adjective'),
 ('목소리', 'Noun')]

In [93]:
train_tokens_okt_norm_stem[0]

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나다', 'Adjective'),
 ('목소리', 'Noun')]

In [122]:
# !pip install soynlp

In [98]:
from pprint import pprint
import sys
sys.path.append('../')

from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer

In [99]:
tokenizer = RegexTokenizer()

sents = [
    '이렇게연속된문장은잘리지않습니다만',
    '숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠',
    '띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠'
]

for sent in sents:
    print('   %s\n->%s\n' % (sent, tokenizer.tokenize(sent)))

   이렇게연속된문장은잘리지않습니다만
->['이렇게연속된문장은잘리지않습니다만']

   숫자123이영어abc에섞여있으면ㅋㅋ잘리겠죠
->['숫자', '123', '이영어', 'abc', '에섞여있으면', 'ㅋㅋ', '잘리겠죠']

   띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠
->['띄어쓰기가', '포함되어있으면', '이정보는', '10', '점', '!', '꼭띄워야죠']



/opt/homebrew/Caskroom/miniforge/base/envs/DL-tf/lib/python3.8/site-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [100]:
tokenizer.tokenize('띄어쓰기가 포함되어있으면 이정보는10점!꼭띄워야죠',flatten=False)

[['띄어쓰기가'], ['포함되어있으면'], ['이정보는', '10', '점', '!', '꼭띄워야죠']]

In [128]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(train['document'].tolist() + test['document'].tolist()) # list of str or like
words = word_extractor.extract()

training was done. used memory 1.038 Gbse memory 0.755 Gb
all cohesion probabilities was computed. # words = 5647
all branching entropies was computed # words = 105708
all accessor variety was computed # words = 105708


In [129]:
len(words)

3836

In [130]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

ㅋㅋ     (6727, 0.973, 5.095)
영화     (57286, 0.941, 4.929)
ㅠㅠ     (2061, 0.958, 4.890)
ㅎㅎ     (1614, 0.925, 4.835)
ㅡㅡ     (1616, 0.958, 4.682)
합니다     (363, 0.750, 4.860)
ㅜㅜ     (705, 0.948, 4.602)
드라마     (5249, 0.858, 4.665)
너무     (14503, 0.970, 4.480)
진짜     (9613, 0.676, 4.838)
차라리     (865, 0.656, 4.772)
입니다     (301, 0.540, 4.919)
된다     (477, 0.646, 4.714)
든다     (248, 0.895, 4.339)
정말     (11882, 0.677, 4.582)
뭔가     (1288, 0.674, 4.584)
역시     (2391, 0.523, 4.831)
솔직히     (1318, 0.898, 4.285)
준다     (447, 0.559, 4.716)
봤는데     (3338, 0.573, 4.671)
까지     (267, 0.402, 5.009)
근데     (990, 0.724, 4.359)
완전     (3135, 0.691, 4.357)
때문에     (918, 0.660, 4.401)
항상     (394, 0.849, 4.138)
계속     (1170, 0.731, 4.202)
ㅉㅉ     (472, 0.983, 3.869)
마지막에     (789, 0.443, 4.662)
함께     (711, 0.764, 4.066)
그리고     (2135, 0.288, 5.037)


In [141]:
scores = {}
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True):
    # scores[word] = score.cohesion_forward
    scores[word] = word_score(score)

In [142]:
scores

{'ㅋㅋ': 158.81836633039003,
 '영화': 130.0173922454327,
 'ㅠㅠ': 127.44209945198614,
 'ㅎㅎ': 116.38622962127909,
 'ㅡㅡ': 103.5018067716247,
 '합니다': 96.7938631591607,
 'ㅜㅜ': 94.44436346879404,
 '드라마': 91.15529783966187,
 '너무': 85.52745994842417,
 '진짜': 85.23054325353004,
 '차라리': 77.46322260892377,
 '입니다': 73.92772504374274,
 '된다': 72.08486954723155,
 '든다': 68.57750388757142,
 '정말': 66.2078033089349,
 '뭔가': 66.00355950646548,
 '역시': 65.52763542176626,
 '솔직히': 65.21595731181061,
 '준다': 62.39722377088773,
 '봤는데': 61.1772329399878,
 '까지': 60.23217772617534,
 '근데': 56.58006394791445,
 '완전': 53.902518020296654,
 '때문에': 53.79905772440471,
 '항상': 53.236084658025064,
 '계속': 48.81391963325982,
 'ㅉㅉ': 47.10496028620962,
 '마지막에': 46.84775704384907,
 '함께': 44.529958610964485,
 '그리고': 44.43281772753415,
 '갈수록': 43.66909354352431,
 '개인적으로': 43.20377179037424,
 '얼마나': 40.875593880759254,
 '갑자기': 40.658330748075535,
 'ㄷㄷ': 40.23880915565518,
 '됩니다': 39.823865320584595,
 '봤습니다': 36.927228691029676,
 '그냥': 36.91

In [143]:
cohesion_scores = word_extractor.all_cohesion_scores()
cohesion_scores['별로'] # (cohesion_forward, cohesion_backward)

all cohesion probabilities was computed. # words = 5647


(0.565947242206235, 0.00426182930827232)

In [144]:
def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

In [145]:
branching_entropy = word_extractor.all_branching_entropy()
branching_entropy['별로'] # (left_branching_entropy, right_branching_entropy)

all branching entropies was computed # words = 105708


(3.6782606986656257, 3.6982209638368735)

In [146]:
accessor_variety = word_extractor.all_accessor_variety()
accessor_variety['별로'] # (left_accessor_variety, right_accessor_variety)

all accessor variety was computed # words = 105708


(121, 149)

In [147]:
tokenizer = MaxScoreTokenizer(scores=scores)
tokenizer.tokenize('ㅋㅋ영화노잼')

['ㅋㅋ', '영화', '노잼']

In [148]:
def tokenize_soy(texts):
    return [tokenizer.tokenize(text) for text in texts]

In [149]:
train_tokens_soy, time_soy = measure_time(tokenize_soy, train_texts)
print(f"Okt norm_stem 실행 시간: {time_soy:.2f} 초")

Okt norm_stem 실행 시간: 0.04 초


In [152]:
train

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [153]:
from soynlp.tokenizer import LTokenizer

ltokenizer = LTokenizer(scores =scores)
train['ltokenizer'] = train['document'].apply(lambda x: ltokenizer.tokenize(x))

In [ ]:
train.head()

In [ ]:
test['ltokenizer'] = test['document'].apply(lambda x: ltokenizer.tokenize(x))